In [2]:
import sys
import numpy as np
import cv2

##입력이미지 불러오기

src = cv2.imread('coins2.jpg')

if src is None :
    print('Image load failed')
    sys.exit()
    
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
blr = cv2.GaussianBlur(gray, (0, 0), 1)

circles = cv2.HoughCircles(blr, cv2.HOUGH_GRADIENT, 1, 50, 
                           param1 = 150, param2 = 40,
                           minRadius=20, maxRadius=80)

##param1 = cannyedge detection 에서 사용함

money = 0
dst = src.copy()
if circles is not None:
    for i in range(circles.shape[1]):
        cx, cy, radius = (circles[0][i]).astype(dtype = 'uint16')
        cv2.circle(dst, (cx, cy), radius, (0, 0, 255), 2, cv2.LINE_AA)
        
        # 
        x1 = int(cx - radius)
        y1 = int(cy - radius)
        x2 = int(cx + radius)
        y2 = int(cy + radius)
        
        radius = int(radius)
        
        crop = src[y1:y2, x1:x2, :]
        ch, cw = crop.shape[:2]
        
        mask = np.zeros((ch, cw), np.uint8)
        cv2.circle(mask, (cw//2, ch//2), radius, 255, -1)
        
       
        
        #mask영역에서만 HUE 계산(40 쉬프트, 평균계산)
        
        hsv = cv2.cvtColor(crop, cv2.COLOR_BGR2HSV)
        hue, _, _ = cv2.split(hsv)
        hue_shift = ((hue+40) % 180) ##hue 최대값 180 더커지면 0으로
        mean_of_hue = cv2.mean(hue_shift, mask)[0]
        
        print(mean_of_hue)
        
        cv2.imshow('crop', crop)
        cv2.imshow('mask', mask)
        cv2.waitKey()
        
        # Hue 평균이 90보다 작으면 10원, 90보다 크면 100원으로 간주
        won = 100
        if mean_of_hue < 90:
            won = 10

        money += won

        cv2.putText(crop, str(won),     (20, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    0.75, (255, 0, 0), 2, cv2.LINE_AA)

cv2.putText(dst, str(money) + ' won', (40, 80),
            cv2.FONT_HERSHEY_DUPLEX, 2, (255, 0, 0), 2, cv2.LINE_AA)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()

cv2.destroyAllWindows()

137.29013995911308
137.01824186192798
137.88781518949116
48.02338634237605
52.345494231368875
141.5134156378601
48.06457293806868
